In [1]:
## Imports
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from datetime import datetime
import os

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [2]:
def get_non_profit_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit
    """

    col_types = {
        "nonprofit_id": "Int64",
        "reporting_year": "Int64",
        "ein": "Int64",
        "businessname": "str",
        "phone": "str",
        "address1": "str",
        "address2": "str",
        "city": "str",
        "stabbrv": "str",
        "zip": "str"
      }

    df = pd.read_csv("../data/nonprofit.txt", sep = "|", dtype=col_types)

    return df
df_np = get_non_profit_df()
df_np = df_np[["nonprofit_id", "stabbrv"]]


In [3]:
def get_non_profit_text_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit_text
    """

    # download_project_maslow_files()÷

    col_types = {
        "nonprofit_text_id": "Int64",
        "reporting_year": "Int64",
        "nonprofit_id": "Int64",
        "grouptype": "str",
        "description": "str"
    }

    df = pd.read_csv("../data/nonprofit_text.txt", sep = "|", encoding="cp1252", dtype=col_types)

    return df

non_agg_df = get_non_profit_text_df()

# get rid of descriptions that are empty
filtered_df = non_agg_df[non_agg_df["description"].str.len() > 0]

df_text = filtered_df.groupby(['nonprofit_id'], as_index = False).agg({'description': ' '.join})

# filter down to TN
df_text = df_text.merge(df_np, on="nonprofit_id")
df_text = df_text[df_text["stabbrv"] == "TN"]

# filter down to aggregated descriptions greater than 10

df_text = df_text[df_text["description"].str.split().str.len() > 10]

In [4]:
len(df_text["nonprofit_id"].unique())

6074

In [5]:
df_text.shape

(6074, 3)

In [6]:
# create a list of the description
data = df_text.description.values.tolist()
np.shape(data)

(6074,)

In [7]:
# tokenize and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
np.shape(data_words)

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(6074,)

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sports', 'program', 'for', 'youth', 'of', 'our', 'community', 'in', 'which', 'they', 'receive', 'sports', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball', 'program', 'powell', 'recreation', 'commission', 'inc']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
try:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
except Exception as e:
    os.system("python -m spacy download en")
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(np.shape(data_lemmatized))
print(data_lemmatized[:2])


/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-pack

(6074,)
[['provide', 'sound', 'supervised', 'provide', 'sound', 'supervised', 'recreational', 'sport', 'program', 'youth', 'community', 'receive', 'sport', 'training', 'beneficial', 'healthy', 'recreation', 'football', 'program', 'baseball'], ['provide', 'food', 'clothng', 'furniture', 'economic', 'challenge', 'emergency', 'purpose', 'great', 'surround', 'area', 'provide', 'help', 'community', 'time', 'disaster', 'economic', 'interuption', 'sudden', 'loss', 'solid', 'rock', 'church', 'porvide', 'food', 'clothing', 'community']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 1), (14, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics, word_topics, phi_values = lda_model.get_document_topics(doc, per_word_topics=True)
        return lda_model.id2word[word_topics[0][0]]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_result"] = get_readable_topic(doc)
    index = index + 1

# export model results into unique file
df_for_csv = df_text[["nonprofit_id", "topic_result", "description"]]
df_for_csv.to_csv(f"{datetime.now().strftime('%Y_%m_%d_%H_%M')}_unique_count_{len(df_text['nonprofit_id'].unique())}_nonprofit_text_output.csv", index=False, sep="|")

In [14]:
len(df_text["nonprofit_id"].unique())

6075

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.709588363242178

Coherence Score:  0.39319723892370206


In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/L

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.247122  0.020112       1        1  23.914060
6      0.099261  0.203251       2        1  15.403687
0      0.227633 -0.162231       3        1  14.040557
1      0.160975 -0.130853       4        1  13.049446
5     -0.111907 -0.178293       5        1  10.079733
2     -0.014835  0.144604       6        1   9.517540
8     -0.164609 -0.047172       7        1   3.954816
3     -0.054539  0.247954       8        1   3.756815
4     -0.206345 -0.127340       9        1   3.151036
9     -0.182756  0.029967      10        1   3.132310, topic_info=         Term         Freq        Total Category  logprob  loglift
95    service  4662.000000  4662.000000  Default  30.0000  30.0000
6     provide  9417.000000  9417.000000  Default  29.0000  29.0000
322    school  2768.000000  2768.000000  Default  28.0000  28.0000
401    county  1462.000000  1462.000000  Default  27.0000  27.0000
381    member  2267.000000  2267.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
82      local   189.065695  1365.134139  Topic10  -4.0527   1.4865
6     provide   317.900213  9417.626092  Topic10  -3.5330   0.0748
139    health   179.625542  2503.923876  Topic10  -4.1039   0.8287
46       fund   121.317032  1277.603571  Topic10  -4.4964   1.1091
3954    crime    63.881188   189.330629  Topic10  -5.1378   2.3769

[486 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1248     10  0.996174       abuse
370       4  0.998479    academic
59        6  0.993953  accomplish
1222      5  0.996881  accordance
272       7  0.991235         act
...     ...       ...         ...
270       4  0.388414        year
270       5  0.099786        year
358       1  0.257928       young
358       4  0.739546       young
14        4  0.998900       youth

[617 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 1, 2, 6, 3, 9, 4, 5, 10])